In [1]:
!python3 -m pip install --upgrade pip

In [2]:
!pip install -q -U google-generativeai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.28.2 which is incompatible.
mediapipe 0.10.15 requires protobuf<5,>=4.25.3, but you have protobuf 5.28.2 which is incompatible.


In [3]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [4]:
# !pip install gradio==3.40.0

In [5]:

# Used to securely store your API key
# from google.colab import userdata

In [5]:
import os
import json
import requests
import gradio as gr
import google.generativeai as genai
from datetime import date
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from typing import Any, Callable, Dict, List, Tuple

load_dotenv(".chatbotenv")

True

In [6]:
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [7]:
# model = genai.GenerativeModel("gemini-1.5-flash")
# response = model.generate_content("Write a story about a magic backpack.")
# print(response.text)

In [9]:
class GPTClient:
    def __init__(self):
        # Initialize the Gemini model
        self.model = genai.GenerativeModel("gemini-1.5-flash")

    def generate_response(self, prompt: str) -> str:
        """
        Generate content using the Gemini API with the specified prompt.
        """
        try:
            response = self.model.generate_content(prompt)
            # 응답에서 차단된 이유를 확인
            if not response.candidates:
                block_reason = getattr(response, 'prompt_feedback', {}).get('block_reason', 'Unknown')
                return f"요청이 차단되었습니다. 차단 이유: {block_reason}"
            return response.text
        except Exception as e:
            return f"오류 발생: {str(e)}"

    def get_args_for_function_call(
        self, messages: List[Dict[str, str]], function_signatures: List[Dict[str, Any]]
    ) -> Dict[str, Any]:
        """
        Return the arguments for the function call or just the response content.
        """
        # Generate a response using the Gemini API
        generated_text = self.generate_response(messages[-1]['content'])

        # Since we are not actually calling a function, return a dictionary with content
        return {
            "content": generated_text,
            "function_call": None  # No function call required
        }

    def request_with_function_call(
        self,
        messages: List[Dict[str, str]],
        function: Callable,
        function_call_resp: Dict[str, Any],
        prompt: str = "",
    ) -> str:
        """
        Process function call response or return generated content directly.
        """
        # If there's no function call in the response, return the content directly
        if not function_call_resp.get("function_call"):
            return function_call_resp["content"]

        function_name = function_call_resp["function_call"]["name"]

        if prompt:
            messages.append({"role": "system", "content": prompt})

        # Simulating external function call
        kwargs = json.loads(function_call_resp["function_call"]["arguments"])
        function_result = function(**kwargs)

        messages.append(function_call_resp)
        messages.append(
            {"role": "function", "name": function_name, "content": function_result}
        )

        # Generating final response with Gemini API
        res = self.generate_response(messages[-1]['content'])
        return res

    def summarize(self, texts: str) -> str:
        prompt = f"""
            Summarize the following sentences:
            ---
            {texts}
            """
        return self.generate_response(prompt)

    def translate(self, texts: str) -> str:
        prompt = f"""
            Translate the following sentences to Korean:
            ---
            {texts}
            """
        return self.generate_response(prompt)

def text_clean(x):
    x = x.replace("&quot;", "").replace("<b>", "").replace("</b>", "").replace("‘", "").replace("’", "")
    return x

gpt_client = GPTClient()

class NewsApiClient:
    def __init__(self):
        self.client_id = os.getenv('client_id')
        self.client_secret = os.getenv('client_secret')
        self.max_num_articles = 5

    def get_articles(self, query: str = None) -> str:
        """Naver News API를 사용하여 검색된 기사를 처리합니다."""
        base_url = "https://openapi.naver.com/v1/search/news.json"
        params = {
            'query': query,
            'display': 100,  # 최대 100개의 기사를 가져오도록 설정
            'start': 1,
            'sort': 'date'
        }
        headers = {
            "X-Naver-Client-Id": self.client_id,
            "X-Naver-Client-Secret": self.client_secret
        }

        # Naver News API 호출
        response = requests.get(base_url, params=params, headers=headers)
        print(response.url)  # 디버깅용으로 URL 출력

        if response.status_code == 200:
            data = response.json()
            print(type(data))  # 데이터 타입 출력 (json 데이터)
        else:
            print("Error Code:", response.status_code)
            return "Error fetching articles"

        today = date.today()
        formatted_date = today.strftime("%a, %d %b %Y")  # 이미지의 날짜 형식에 맞게 포맷

        result = {
            "title": [],
            "originallink": [],
            "link": [],
            "description": [],
            "pubDate": []
        }

        # 검색된 기사 중 오늘 날짜에 해당하는 기사만 추출
        for item in data['items']:
            if formatted_date not in item['pubDate']:
                # 각각의 컬럼에 맞는 데이터를 result 딕셔너리에 추가
                result['title'].append(text_clean(item['title']))
                result['originallink'].append(item.get('originallink', 'N/A'))
                result['link'].append(item['link'])
                result['description'].append(text_clean(item['description']))
                result['pubDate'].append(item['pubDate'])
            else:
                break

        # 처리된 결과를 JSON 형식으로 반환
        return json.dumps(result, ensure_ascii=False, indent=4)


news_api_client = NewsApiClient()

def respond(prompt: str, chat_history: List[str]) -> Tuple[str, List[str], gr.Dropdown, str]:
    """
    사용자 입력(prompt)에 따라 Naver 뉴스 API에서 기사를 검색하고,
    그 결과를 사용자에게 출력하고, 기사 제목을 드롭다운에 표시하는 함수.
    또한 기사 데이터(articles_json)를 상태로 반환하여 나중에 사용.
    """
    # 사용자 입력에 따라 메시지 생성
    messages = [{"role": "user", "content": prompt}]

    # GPT에서 응답 생성
    answer = gpt_client.generate_response(prompt)

    # Naver 뉴스 API에서 기사를 검색하는 부분
    articles_json = news_api_client.get_articles(query=prompt)
    articles = json.loads(articles_json)  # JSON 데이터를 딕셔너리로 변환

    # 기사 제목 리스트 생성
    title_list = articles['title'] if 'title' in articles else []

    if not title_list:
        chat_history.append((prompt, "관련 기사를 찾을 수 없습니다."))
        return "", chat_history, gr.Dropdown.update(choices=[]), ""

    # 채팅 기록 업데이트
    chat_history.append((prompt, answer))

    # 기사 제목 리스트를 드롭다운에 반영 및 기사 데이터를 상태로 반환
    return "", chat_history, gr.Dropdown.update(choices=title_list), articles_json


def scrap_article(title: str, articles_json: str) -> Tuple[str, str]:
    """
    Naver 뉴스 기사 제목에 맞는 URL을 통해 기사를 가져와 요약하고 번역합니다.
    `articles_json`은 get_articles 함수에서 반환된 기사 정보입니다.
    """
    if not articles_json:
        return "기사 데이터를 가져오는 중 오류가 발생했습니다.", ""

    # JSON 데이터를 딕셔너리로 변환
    articles = json.loads(articles_json)

    # title을 기반으로 articles에서 해당 기사의 URL을 찾아서 가져오기
    if title in articles['title']:
        index = articles['title'].index(title)  # 선택된 제목의 인덱스를 찾음
        url = articles['originallink'][index]  # 해당 제목의 originallink 가져오기
    else:
        return "해당 제목에 대한 기사를 찾을 수 없습니다.", ""

    # 기사 페이지에서 내용을 가져오기
    rep = requests.get(url)
    if rep.status_code != 200:
        return "기사를 가져오는 중 오류가 발생했습니다.", ""

    # BeautifulSoup을 사용해 본문 파싱
    soup = BeautifulSoup(rep.content, "html.parser")

    # 기사 본문 추출
    article = ""
    for paragraph in soup.find_all("p"):  # 본문이 <p> 태그로 구성되어 있다고 가정
        article += paragraph.text.strip()

    if not article:
        return "기사를 스크랩하는 중 오류가 발생했습니다.", ""

    # Gemini API를 사용해 기사를 요약하고 번역
    summarized_article = gpt_client.summarize(article)
    translated_article = gpt_client.translate(summarized_article)

    return summarized_article, translated_article

def clear_output() -> Tuple[str, List[str], gr.Dropdown, str, str, str]:
    """
    Clear 버튼을 눌렀을 때 모든 출력 요소들을 초기화하는 함수.
    """
    return "", [], gr.Dropdown.update(choices=[], value=None), "", "", ""  # article_list도 완전히 초기화 (value=None 추가)

# Gradio 인터페이스 설정
with gr.Blocks() as app:
    gr.Markdown("# 뉴스 기사 탐색 챗봇")

    with gr.Row():
        with gr.Column():
            gr.Markdown("## Chat 얻고 싶은 정보에 대해 질문해보세요.")
            chatbot = gr.Chatbot(label="Chat History")
            prompt = gr.Textbox(label="Input prompt")
            clear = gr.Button("Clear")  # Clear 버튼 추가

        with gr.Column():
            gr.Markdown("## Select News article 원하는 기사를 선택하세요.")
            article_list = gr.Dropdown(label="Article List", choices=[])
            abstract_box = gr.Textbox(label="Summarized article", lines=10, interactive=False)
            translate_box = gr.Textbox(label="Translated article", lines=10, interactive=False)
            scrap_btn = gr.Button("Get article!")

    # 상태로 articles_json을 유지하도록 설정
    articles_state = gr.State()

    # 사용자 입력 시 기사 제목을 가져와 article_list에 업데이트, 기사 데이터를 상태에 저장
    prompt.submit(respond, [prompt, chatbot], [prompt, chatbot, article_list, articles_state])

    # 사용자가 기사 선택 후 "Get article!" 버튼을 누르면 해당 기사를 요약 및 번역
    scrap_btn.click(scrap_article, inputs=[article_list, articles_state], outputs=[abstract_box, translate_box])

    # Clear 버튼 클릭 시 모든 출력 초기화 (기사 내용까지 포함)
    clear.click(clear_output, outputs=[prompt, chatbot, article_list, articles_state, abstract_box, translate_box])

app.launch(inline=False, share=True)

Running on local URL:  http://127.0.0.1:7867
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://6c644b13c083ca7f68.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


https://openapi.naver.com/v1/search/news.json?query=%EB%AF%B8%EA%B5%AD+%EB%8C%80%EC%84%A0%EC%97%90+%EB%8C%80%ED%95%B4%EC%84%9C+%EC%95%8C%EB%A0%A4%EC%A4%98&display=100&start=1&sort=date
<class 'dict'>


In [ ]:
app.close()

Closing server running on port: 7860
